In [1]:
!pip install -U transformers

In [2]:
!pip install -U bitsandbytes

In [3]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
from PIL import Image

model_id = "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit"
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.float16
)
processor = AutoProcessor.from_pretrained(model_id)

# 안전한 토큰 설정
eos_id = processor.tokenizer.eos_token_id
im_end_id = processor.tokenizer.convert_tokens_to_ids("<|im_end|>")
stop_ids = [t for t in [eos_id, im_end_id] if t is not None]

if model.generation_config.eos_token_id is None:
    model.generation_config.eos_token_id = eos_id
if model.generation_config.pad_token_id is None:
    # Qwen 계열은 pad_token이 없을 수 있어 eos로 대체
    model.generation_config.pad_token_id = eos_id

def chat(messages, max_new_tokens=768, temperature=0.7, top_p=0.9):
    # Qwen2.5 전용 템플릿
    prompt = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # 이미지 로딩 (RGB 권장)
    imgs = []
    for msg in messages:
        for c in msg["content"]:
            if c["type"] == "image":
                imgs.append(Image.open(c["image"]).convert("RGB"))

    # 텐서화
    inputs = processor(
        text=prompt,
        images=imgs if imgs else None,
        return_tensors="pt"
    ).to(model.device)

    input_len = inputs["input_ids"].shape[-1]

    # 생성
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,          # 목록형 설명이 끊기지 않도록 샘플링 권장
        temperature=temperature,
        top_p=top_p,
        eos_token_id=stop_ids,   # eos 혹은 <|im_end|> 도달 시 종료
        pad_token_id=model.generation_config.pad_token_id
    )

    # "생성된 부분만" 디코드
    new_tokens = gen_ids[0, input_len:]
    text = processor.decode(new_tokens, skip_special_tokens=True).strip()

    return text

# 시나리오 1
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "image": "/content/사진 5.png"},
        {"type": "text", "text": "I want to pay."}
    ]
}]

answer = chat(conversation)
print(answer)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


It looks like you're in a workspace with various items around, including papers, a fan, and what appears to be a green tumbler. However, there is no visible payment method or point of contact for making a payment in the image provided. If you need to make a payment, you might consider using a digital payment platform or contacting the service provider directly if this is a business-related transaction. If it's personal, ensure that you have the correct payment method and details ready before proceeding.


In [8]:
# 시나리오 2
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "image": "/content/사진 6.png"},
        {"type": "text", "text": "My mouth smells weird."}
    ]
}]

answer = chat(conversation)
print(answer)

It's possible that the smell coming from your mouth could be due to several factors, including:

1. **Food**: If you've recently eaten something strong-smelling or spicy, this can sometimes linger in your mouth.
2. **Bad Breath (Halitosis)**: This is a common condition where your breath has an unpleasant odor. It can be caused by various factors such as poor dental hygiene, certain foods, and even some medical conditions.
3. **Medications**: Some medications can cause a metallic taste or bad breath.
4. **Dehydration**: Not drinking enough water can lead to dry mouth, which can make your breath smell bad.
5. **Stress**: High levels of stress can also contribute to bad breath.

If the smell persists or if it's accompanied by other symptoms like a sore throat or a persistent cough, it might be wise to consult a healthcare professional for a more thorough evaluation. They can help determine the underlying cause and provide appropriate treatment if necessary.
